# 1 Preparación de los datos para entrenar nuestros modelos


Los datos fueron compartidos a través de una colaboración en Archivo Historico del Municipio de Centro, los cuales tenian la siguente estructura

![datos](https://i.imgur.com/U9WOf8p.png)

Cada carpeta corresponde a un libro de acta de cabildo diferente, estando organizado de la siguente manera

![organizacion](https://i.imgur.com/kc9wGZZ.png)

En total se tiene mas de 3000 imagenes diferentes. Cada imagen corresponde a un par de páginas de los de sesiones de cabildo. Nuestra primera tarea fue extraer todas las imagenes de cada carpeta para posteriormente separarlas en dos carpetas, imagenes en buen estado e imagenes a resturar.

![separación](https://i.imgur.com/upRB1eq.png)

Podemos ver que tenemos separado esto en dos conjuntos. 

-----

# Creación del dataset de entrenamiento

La forma en que la red neuronal va mejorando es a traves de dos procesos llamados "Feedforward" y "Backpropagation" que en resumen con los datos de entrada genera un resultado y con el resulto mejora los parametros de la red neuronal hasta obtener el resultado deseado. La función "Loss" será la que evaluará el desempeño de nuestro modelo. 

![digrama](https://i.imgur.com/Du8TZSv.png)

Para el tipo de red neural que desarrollaremos, esta evaluación se realiza pixel por pixel, por lo que requerimos, por asi decirlo, dos imagenes espejo en cuanto a las carecteristicas buenas, una en el estado más óptimo posible y otra en un destado deteriorado, es decir, que alguna de los siguentes defectos: manchas, arrugas, palabras duplicadas en la impresión, oxido, sellos, marcas de páginas anteriores, desenfoque, baja legibilidad. 